# NMF analysis on calcium movies

In [1]:
import os

# standard libraries
import numpy as np
# import matplotlib.pyplot as plt
from sklearn.decomposition import NMF

# custom library
from patchnmf.data_io import get_tiff, get_save_path, export_conts_fiji
from patchnmf.analyse.compute import downsample_tiff_avg, compute_nmfpx_blur_thr, get_thr_img_auto, get_roi_conts, get_loading_times 
from patchnmf.plot import plot_nmf_t, plot_nmfpx_blur_thr, plot_rois_overlay, plot_roi_conts_largest, plot_roi_area_hist, plot_px_nmf_corr, plot_roi_loading_time

# going to root directory (if not there yet)
current_dir = os.getcwd().split('/')[-1]
if current_dir != 'barrel-patch_analysis': 
    os.chdir('..')

# reload code if library changes
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

## Setting parameters

In [2]:
# defining analysis parameters
# ds = 'ani98_XXXX-XX-XX_a' # in this case set bruker to false 

# Normally this would just be the dataset name as per deve-networks convention, 
# but here its the raw TSeries subfolder + additionally sometimes there are 
# multiple TSeries per dataset name (it would be ideal if this wasnt the case) 
# so we need to specify the full path to the raw TSeries
bruker = True
ds = 'ani51_2023-04-20/TSeries-03282023-1355_940_135umdeep-002'

n_components = 20
blur_std = 6.5
downs_fact = 4 # keep as 1 for no downsampling IMPORTANT: downsampling will mean fewer significant components in cross-validation
resolution = 1.2 # in um 

## Loading and pre-processing tiff

In [ ]:
if bruker:
    tiff = 
    
else:
    tiff = get_tiff(ds)
    

In [3]:
tiff = downsample_tiff_avg(tiff, n=downs_fact) # downsample (to speed up cross-validation) -> not done if downs_fact = 1

FileNotFoundError: [Errno 2] No such file or directory: 'data/ani98_XXXX-XX-XX_a/suite2p/plane0/reg_tif'

In [ ]:
# flattening movie to input it to NMF (Negative *Matrix* Factorisation works on matrices, not tensors like movies)
# but its not problem because the results that we get can then be easily reshaped back into an x by y frame (FOV)
tiff_flat = np.reshape(tiff, (tiff.shape[0] , tiff.shape[1]*tiff.shape[2]))
print(f'Shape of video as a matrix (input to NMF): {tiff_flat.shape}') 

# NMF on pixels

In [ ]:
#initialising nmf and fitting to pixels
nmf_px = NMF(n_components=n_components)
nmf_px.fit(tiff_flat);

In [ ]:
# here we get the raw NMF component on pixels, raw NMF with gaussin blur and an automatically thresholded blurred NMF (binary 'ROI')
loading_imgs, loading_imgs_filt, rois_auto = compute_nmfpx_blur_thr(nmf_px, tiff.shape, blur_std=blur_std)

In [ ]:
plot_nmfpx_blur_thr(loading_imgs, loading_imgs_filt, rois_auto)

In [ ]:
plot_rois_overlay(rois_auto, tiff.shape) # this will look ugly for too many components, also probably not the best viusalisation

In [ ]:
# this is a function that takes ROIs as a binary image (as above) and outputs the coordinates of points that would encircle them
conts, n_conts = get_roi_conts(rois_auto)

In [ ]:
plot_roi_conts_largest(conts, tiff.shape)

In [ ]:
plot_roi_area_hist(rois_auto, n_bins=10, resolution=resolution)

In [ ]:
# looking at correlation (in the binary case this is somewhat equivalent to the percentage of overlap)
# NOTE: this plot is deceiving because it only shows positive correlations - it saturates the negative ones
plot_px_nmf_corr(nmf_px)

# NMF on time

In [ ]:
# Initialising nmf and fitting to time
nmf_t = NMF(n_components=n_components)
nmf_t.fit(tiff_flat.T);

In [ ]:
loading_times = get_loading_times(nmf_t)

In [ ]:
plot_roi_loading_time(rois_auto, loading_times)

# Saving variables

In [ ]:
# saving ROIs as text files to import in FIJI
save_path = get_save_path(ds)
export_conts_fiji(conts, save_path)

In [ ]:
# variables to save
tiff_dimred_exp = {'loading_imgs': loading_imgs,
                   'loading_times': loading_times,
                   'rois_auto': rois_auto,
                   'conts': conts}

In [ ]:
np.save(f'{save_path}/export_tiff_dimred.npy', tiff_dimred_exp, allow_pickle=True)